<a href="https://colab.research.google.com/github/kshitijsinghh/IPl-Analysis-Using-SparkSQL/blob/main/IPL_Analysis_By_KshitijSingh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Uploading Datasets in Databricks

In [ ]:
%python
ipl_match = spark.read.format("csv").load("dbfs:/FileStore/shared_uploads/kshitijsinghh99@gmail.com/ipl_matches-2.csv",header="true")
ipl_venues = spark.read.format("csv").load("dbfs:/FileStore/shared_uploads/kshitijsinghh99@gmail.com/ipl_venue-3.csv",header="true")
ipl_balls = spark.read.format("csv").load("dbfs:/FileStore/shared_uploads/kshitijsinghh99@gmail.com/ipl_ball_by_ball.csv",header="true")

ipl_match.createOrReplaceTempView("ipl_matches")
ipl_venues.createOrReplaceTempView("ipl_venue")
ipl_balls.createOrReplaceTempView("ipl_ball")

Q2.1-Find the top 3 venues which hosted the most number of eliminator matches? 

In [ ]:
%sql
with cte as(
select count(ipl_matches.match_id) as number_of_matches , ipl_venue.venue from ipl_matches inner join ipl_venue on ipl_matches.venue_id= ipl_venue.venue_id where ipl_matches.eliminator= 'Y' group by ipl_venue.venue 
)
,cts as(
select * , rank() over (order by number_of_matches desc) as rank from cte 
)
select cts.number_of_matches, cts.venue from cts where rank<4

number_of_matches,venue
3,Dubai International Cricket Stadium
2,Sheikh Zayed Stadium
1,"Sardar Patel Stadium, Motera"
1,Feroz Shah Kotla
1,"Rajiv Gandhi International Stadium, Uppal"
1,M.Chinnaswamy Stadium
1,Wankhede Stadium
1,Newlands
1,Saurashtra Cricket Association Stadium
1,"MA Chidambaram Stadium, Chepauk"


Q2.2-Return the most number of catches taken by a player in IPL history?

In [ ]:
%sql
with cte as
(select sum(ipl_ball.is_wicket) as catches , ipl_ball.fielder from ipl_ball where ipl_ball.is_wicket=1 and ipl_ball.dismissal_kind= 'caught' group by ipl_ball.fielder
)
,cts as(
select * , dense_rank() over (order by catches desc) as rank from cte 
)
select catches, fielder from cts where rank=1

catches,fielder
118.0,KD Karthik


Q2.3-Highest wicket taker in matches which were affected by Duckworth-Lewis’s method (D/L method).

In [ ]:
%sql
with cte as(
select count(ipl_ball.is_wicket) as wicket, ipl_ball.bowler as bowler from ipl_ball inner join ipl_matches on ipl_ball.match_id = ipl_matches.match_id where ipl_matches.method = 'D/L' and ipl_ball.is_wicket=1 group by bowler
)
,cts as(
select *, dense_rank() over(order by wicket desc) as rank from cte
)
select wicket, bowler,rank from cts where rank=1

wicket,bowler,rank
8,R Ashwin,1
8,AB Dinda,1


Q2.4-Highest strike rate by a batsman in non powerplay overs(7-20 overs)

In [ ]:
%sql
with cte as(
select sum(ipl_ball.batsman_runs) as runs, count(ipl_ball.ball) as balls, ipl_ball.batsman as batsman from ipl_ball where ipl_ball.overs >6 group by batsman 
)
,cts as(
select ((runs/balls)*100) as strike_rate , batsman from cte
)
,cta as (
select *, dense_rank() over (order by strike_rate desc)as rank from cts)
select batsman, strike_rate from cta where rank=1

batsman,strike_rate
B Stanlake,250.0


Q2.5-Write a query to return a report for highest extra runs in a venue

In [ ]:
%sql
with cte as(
select ipl_matches.match_id as id , ipl_matches.venue_id as Vid, ipl_venue.venue as venue, ipl_venue.city as city from ipl_matches inner join ipl_venue on ipl_matches.venue_id=ipl_venue.venue_id
)
,cts as (
select cte.id, cte.Vid, cte.venue as venue, cte.city as city , ipl_ball.extra_runs as extra_runs from cte inner join ipl_ball on cte.id=ipl_ball.match_id
)
,cta as(
select sum(cts.extra_runs) as extra_runs , cts.venue, cts.city from cts group by venue ,city
)
,ctb as(
select *, rank() over(order by extra_runs desc) as rank from cta
)
select * from ctb where rank=1


extra_runs,venue,city,rank
1355.0,M.Chinnaswamy Stadium,Bengaluru,1


Q2.6-Cricketers with the most number of players of the match award in neutral venues.

In [ ]:
%sql
with cte as(
select ipl_matches.player_of_match as player_name , count(neutral_venue) as number_of_awards from ipl_matches where ipl_matches.neutral_venue = 1 group by player_name 
)
,cts as(
select *, dense_rank() over (order by number_of_awards desc) as rank from cte
)
select * from cts where rank=1

player_name,number_of_awards,rank
YK Pathan,3,1
GJ Maxwell,3,1
JH Kallis,3,1
JP Duminy,3,1


Q2.7-List of top 10 players with the highest batting average

In [ ]:
%sql
with cte as (
select sum(ipl_ball.batsman_runs) as total_runs, sum(ipl_ball.is_wicket) as total_out,(sum(ipl_ball.batsman_runs)/sum(ipl_ball.is_wicket)) as batting_avg , ipl_ball.batsman as batsman from ipl_ball  group by batsman
)
,cts as (
select * , rank() over (order by batting_avg desc) as r from cte 
)
select * from cts where r<11

total_runs,total_out,batting_avg,batsman,r
88.0,1.0,88.0,Iqbal Abdulla,1
204.0,3.0,68.0,RD Gaikwad,2
167.0,3.0,55.666666666666664,MN van Wyk,3
203.0,4.0,50.75,PD Collingwood,4
577.0,13.0,44.38461538461539,HM Amla,5
44.0,1.0,44.0,LH Ferguson,6
2647.0,62.0,42.693548387096776,KL Rahul,7
4849.0,114.0,42.53508771929825,AB de Villiers,8
5254.0,126.0,41.698412698412696,DA Warner,9
2029.0,49.0,41.40816326530612,JP Duminy,10


Q2.8-Query to find out who has officiated (as an umpire) the most number of matches in IPL.

In [ ]:
%sql
with cte as(
select match_id, stack(2,'umpire_1',umpire1,'umpire_2',umpire2) as (umpire_term,umpire_name) from ipl_matches 
)
,cts as(
select count(cte.match_id) as number_of_matches, cte.umpire_name as umpire_name from cte group by cte.umpire_name
)
,cta as(
select *, rank() over (order by number_of_matches desc) as rank from cts
)
select * from cta where rank=1

number_of_matches,umpire_name,rank
121,S Ravi,1


Q2.9-How winning/losing tosses can impact a match and it's result?

In [ ]:
%sql

select count(*) as Counts, 'Number_of_Times_Toss_winner_is_match_winner' as Status from ipl_matches where ipl_matches.toss_winner=ipl_matches.winner
union all

select count(*) as Counts, 'Number_of_Times_Toss_winner_is_not_match_winner' as Status from ipl_matches where ipl_matches.toss_winner!=ipl_matches.winner




Counts,Status
418,Number_of_Times_Toss_winner_is_match_winner
398,Number_of_Times_Toss_winner_is_not_match_winner


Uploading Datasets in Sqlite3

In [ ]:
%python
# coding=utf8
import sqlite3 as sql
import pandas as pd

ipl_match = spark.read.format("csv").load("dbfs:/FileStore/shared_uploads/kshitijsinghh99@gmail.com/ipl_matches-2.csv",header="true")
ipl_venues = spark.read.format("csv").load("dbfs:/FileStore/shared_uploads/kshitijsinghh99@gmail.com/ipl_venue-3.csv",header="true")
ipl_balls = spark.read.format("csv").load("dbfs:/FileStore/shared_uploads/kshitijsinghh99@gmail.com/ipl_ball_by_ball.csv",header="true")


ipl_matches = ipl_match.toPandas()
ipl_venue = ipl_venues.toPandas()
ipl_ball = ipl_balls.toPandas()
conn = sql.connect('IPL_Analysis.db')
ipl_matches.to_sql('ipl_matches',conn)
ipl_venue.to_sql('ipl_venue',conn)
ipl_ball.to_sql('ipl_ball',conn)




Querying in Sqlite3

In [ ]:
%python 
# coding=utf8
import sqlite3 as sql
class Database:
    def connection(self):
        try:
            con = sql.connect('IPL_Analysis.db')
            print("Connected To Database")
            
            return con
        except Exception as e:
            print(e)
    
    
    
        
    def query1(self):
        con =self.connection()
        
        query= con.cursor()
        query.execute('with cte as(select count(ipl_matches.match_id) as number_of_matches , ipl_venue.venue from ipl_matches inner join ipl_venue on ipl_matches.venue_id= ipl_venue.venue_id where ipl_matches.eliminator= \'{}\' group by ipl_venue.venue ),cts as(select * , rank() over (order by number_of_matches desc) as rank from cte ) select cts.number_of_matches, cts.venue from cts where rank<\'{}\''.format("Y","4"))
        
        data = query.fetchall()
        for i in data:
            print("Number of matches hosted- "+str(i[0])+" ,"+"Name of stadium- "+str(i[1]))
        
        
        
        
    def query2(self):
        con =self.connection()
        print("connected")
        
        
        query= con.cursor()
        query.execute('with cte as(select sum(ipl_ball.is_wicket) as catches , ipl_ball.fielder from ipl_ball where ipl_ball.is_wicket=\'{}\' and ipl_ball.dismissal_kind= \'{}\' group by ipl_ball.fielder),cts as(select * , rank() over (order by catches desc) as rank from cte ) select catches,fielder  from cts limit \'{}\' ' .format('1','caught',"1"))
        data = query.fetchall()
        for i in data:
            print("Number of catches- "+str(i[0])+" ,"+"Name of player- "+str(i[1]) )
        
        
    def query3(self):
        
        con =self.connection()
        query= con.cursor()
        query.execute('with cte as(select sum(ipl_ball.is_wicket) as wicket, ipl_ball.bowler as bowler from ipl_ball inner join ipl_matches on ipl_ball.match_id = ipl_matches.match_id where ipl_matches.method = \'{}\'  and ipl_ball.is_wicket=\'{}\' group by bowler),cts as(select *, dense_rank() over(order by wicket desc) as rr from cte) select cts.wicket, cts.bowler,cts.rr from cts  limit \'{}\' '.format("D/L","1","2"))
        
        data = query.fetchall()
        for i in data:
            print("Number of wickets- "+str(i[0])+", "+"Name of player- "+ str(i[1])+" ")
        
        
    
    def query4(self):
        con =self.connection()
        
        query= con.cursor()
        query.execute('with cte as(select sum(ipl_ball.batsman_runs) as runs, count(ipl_ball.ball) as balls, ipl_ball.batsman as batsman from ipl_ball where ipl_ball.overs >\'{}\' group by batsman ),cts as(select ((runs/balls)*100) as strike_rate , batsman from cte),cta as (select *, dense_rank() over (order by strike_rate desc)as r from cts) select batsman, strike_rate from cta where r<\'{}\'  '.format("6","2"))
        data = query.fetchall()
        for i in data:
            print("Name of Player- "+str(i[0])+", "+"Strike_rate- "+str(i[1]))
        
        
    def query5(self):
        con =self.connection()
        
        query= con.cursor()
        query.execute('with cte as(select ipl_matches.match_id as id , ipl_matches.venue_id as Vid, ipl_venue.venue as venue, ipl_venue.city as city from ipl_matches inner join ipl_venue on ipl_matches.venue_id=ipl_venue.venue_id),cts as (select cte.id, cte.Vid, cte.venue as venue, cte.city as city , ipl_ball.extra_runs as extra_runs from cte inner join ipl_ball on cte.id=ipl_ball.match_id),cta as(select sum(cts.extra_runs) as extra_runs , cts.venue, cts.city from cts group by venue ,city),ctb as(select *, rank() over(order by extra_runs desc) as rank from cta)select * from ctb limit \'{}\' ' .format("1"))
        data = query.fetchall()
        for i in data:
            print("Extras_Run- "+str(i[0])+", "+"Stadium_Name- "+str(i[1])+", "+"Stadium_City- "+str(i[2]))
        
        
    def query6(self):
        con =self.connection()
        
        query= con.cursor()
        query.execute('with cte as(select ipl_matches.player_of_match as player_name , count(neutral_venue) as number_of_awards from ipl_matches where ipl_matches.neutral_venue = 1 group by player_name ),cts as(select *, dense_rank() over (order by number_of_awards desc) as rank from cte)select * from cts limit \'{}\'' .format("4"))
        data = query.fetchall()
        for i in data:
            print("Name of Player- "+str(i[0])+", "+"Number of Awards-"+str(i[1]))
        
        
    def query7(self):
        con =self.connection()
        
        query= con.cursor()
        query.execute('with cte as (select sum(ipl_ball.batsman_runs) as total_runs, sum(ipl_ball.is_wicket) as total_out,(sum(ipl_ball.batsman_runs)/sum(ipl_ball.is_wicket)) as batting_avg , ipl_ball.batsman as batsman from ipl_ball  group by batsman),cts as (select * , dense_rank() over (order by batting_avg desc) as r from cte )select * from cts where r<\'{}\''.format("11"))
        data = query.fetchall()
        for i in data:
            print("Runs_scored- "+ str(i[0])+", "+"Total number of outs- " + ", " + str(i[1]) + ", " + "Batting Average- " + str(i[2])+ ", " + "Name of player- " +str(i[3]))
        
        
    def query8(self):
        con =self.connection()
        
        query= con.cursor()
        query.execute('with cte as(select match_id, stack(\'{}\',\'{}\',\'{}\',\'{}\',\'{}\') as (\'{}\',\'{}\') from ipl_matches ),cts as(select count(cte.match_id) as number_of_matches, cte.umpire_name as umpire_name from cte group by cte.umpire_name),cta as(select *, rank() over (order by number_of_matches desc) as rank from cts)select * from cta where rank=\'{}\''.format("2","umpire_1","umpire1","umpire_2","umpire2","umpire_term","umpire_name","1"))
        data = query.fetchall()
        
        return data
Query= Database()
Query.query1()

Connected To Database
Number of matches hosted- 3 ,Name of stadium- Dubai International Cricket Stadium
Number of matches hosted- 2 ,Name of stadium- Sheikh Zayed Stadium
Number of matches hosted- 1 ,Name of stadium- Feroz Shah Kotla
Number of matches hosted- 1 ,Name of stadium- M.Chinnaswamy Stadium
Number of matches hosted- 1 ,Name of stadium- MA Chidambaram Stadium, Chepauk
Number of matches hosted- 1 ,Name of stadium- Newlands
Number of matches hosted- 1 ,Name of stadium- Rajiv Gandhi International Stadium, Uppal
Number of matches hosted- 1 ,Name of stadium- Sardar Patel Stadium, Motera
Number of matches hosted- 1 ,Name of stadium- Saurashtra Cricket Association Stadium
Number of matches hosted- 1 ,Name of stadium- Wankhede Stadium
